In [1]:
# Descargamos el dataset
!gdown --id 1u1MRRWnjoi3p5eWV6bOtJX4_bZ0DT4eu
!unzip balanced_dataset.zip

Se truncaron las últimas líneas 5000 del resultado de transmisión.
  inflating: balanced_dataset_default/data/idctjG4MjJwEA_video.jpg  
  inflating: balanced_dataset_default/data/idCTJkXenJkCk_video.jpg  
  inflating: balanced_dataset_default/data/idcTKbRC050UA_video.jpg  
  inflating: balanced_dataset_default/data/idctl16uxL9nY_video.jpg  
  inflating: balanced_dataset_default/data/idCTmEDYd2C1w_video.jpg  
  inflating: balanced_dataset_default/data/idcTN5X6bV1J8_video.jpg  
  inflating: balanced_dataset_default/data/idcu0CtwVMKaY_video.jpg  
  inflating: balanced_dataset_default/data/idcu0DPPhRSI0_video.jpg  
  inflating: balanced_dataset_default/data/idcU0DS2m-Glg_video.jpg  
  inflating: balanced_dataset_default/data/idcU0E7eDWYMw_video.jpg  
  inflating: balanced_dataset_default/data/idCU0eC-bR1RU_video.jpg  
  inflating: balanced_dataset_default/data/idCU0ehzT9Bbg_video.jpg  
  inflating: balanced_dataset_default/data/idCu0EN1CpWzU_video.jpg  
  inflating: balanced_dataset_defaul

In [2]:
# Librerias utilizadas.
import pandas as pd
import numpy as np
import tensorflow as tf
from skimage import io
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import Sequence
from tensorflow.keras.utils import to_categorical
import random
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Input,MaxPooling2D, Dropout

In [3]:

# Generador de datos
class DataGenerator(Sequence):

  # imgIDs: Lista de las ID de las imagenes a utilizar en este generator.
  # labels: Diccionario de los labels para cada ID (sin one hot encoding)
  # batch_size: Tamaño de cada batch que se genera por llamado.
  # dim: Dimensiones de las imagenes.
  # n_channels: Numero de canales de la imagen.
  # n_classes: Numero de labels distintos.
  # shuffle: Si revolver los datos o no.
  def __init__(self, imgIDs, labels, batch_size=32, dim=(180, 320), n_channels=3, n_classes=4, shuffle=True):
    self.imgIDs = imgIDs
    self.labels = labels
    self.batch_size = batch_size
    self.dim = dim
    self.n_channels = n_channels
    self.n_classes = n_classes
    self.shuffle = shuffle
    self.on_epoch_end()

  # Retorna el numero de batches del generator.
  def __len__(self):
    return int(np.floor(len(self.imgIDs) / self.batch_size))

  # Retorna un batch.
  def __getitem__(self, index):
    indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
    list_IDs_temp = [self.imgIDs[k] for k in indexes]
    X, y = self.__data_generation(list_IDs_temp)

    return X, y

  # Cuando se termina un epoch, se escogen los siguientes ID.
  def on_epoch_end(self):
    self.indexes = np.arange(len(self.imgIDs))
    if self.shuffle == True:
      np.random.shuffle(self.indexes)

  # Se cargan los datos a medida que se necesitan.
  def __data_generation(self, list_IDs_temp):
    X = np.empty((self.batch_size, *self.dim, self.n_channels))
    y = np.empty((self.batch_size), dtype=int)

    for i, ID in enumerate(list_IDs_temp):
      X[i,] = io.imread("./balanced_dataset_medium/data/" + ID + ".jpg")
      y[i] = self.labels[ID]
    
    X = X.astype("float32") / 255

    return X, to_categorical(y, num_classes=self.n_classes)

In [4]:
# Observamos los datos del CSV.
dataInfo = pd.read_csv("./balanced_dataset_medium/data_info.csv")
dataInfo

,Unnamed: 0,ID,part,label,views,comments,likes,dislikes,topicID
0,0,id---2pGwkL7M_video,part1,0,3543,11.0,63.0,0.0,"['/m/04rlf', '/m/04rlf']"
1,1,id---9beuLW7Y_video,part1,0,1543,0.0,0.0,0.0,"['/m/098wr', '/m/098wr']"
2,2,id---dQKphvYc_video,part1,0,4065,4.0,13.0,0.0,"['/m/04rlf', '/m/064t9']"
3,3,id---EnHBffak_video,part1,0,7808,5.0,36.0,5.0,"['/m/0bzvm2', '/m/0bzvm2']"
4,4,id---FI9JLZRg_video,part1,0,1308,2.0,3.0,4.0,"['/m/0bzvm2', '/m/0bzvm2', '/m/0403l3g']"
...,...,...,...,...,...,...,...,...,...
19980,19980,idE2gK1mLbJKE_video,part189,3,3547841,460.0,16357.0,777.0,"['/m/04rlf', '/m/01lyv']"
19981,19981,ide2hvBdWy0X8_video,part189,3,1414097,568.0,8945.0,385.0,"['/m/04rlf', '/m/064t9', '/m/0glt670']"
19982,19982,ide2jPLqvMRwM_video,part189,3,1884647,112.0,2071.0,853.0,['/m/04rlf']
19983,19983,ide2JR9LtAic0_video,part189,3,1459441,3169.0,20779.0,386.0,"['/m/025zzc', '/m/0bzvm2', '/m/0bzvm2', '/m/02..."


In [5]:
# Tamaño de los grupos, donde se indica con proporciones.
# El resto se le asigna al grupo de test.
trainSize = 0.6
valSize = 0.2

# Generamos el diccionario de los labels, y cargamos todos los IDs en una lista.
labels = {}
data = []

for i, row in dataInfo.iterrows():
  data.append(row["ID"])
  labels[row["ID"]] = row["label"]

# Revolvemos la lista de ID.
random.Random(42).shuffle(data)

# Generamos las listas de IDs para cada grupo.
trainData = data[:int(len(data)*trainSize)]
valData = data[int(len(data)*trainSize) : int(len(data)*trainSize) + int(len(data)*valSize)]
testData = data[int(len(data)*trainSize) + int(len(data)*valSize):]

# Creamos el diccionario de listas de ID para cada grupo.
partition = {}
partition["train"] = trainData
partition["validation"] = valData
partition["test"] = testData

In [6]:
# TODO: Graficar una wea

In [7]:
# Creamos los generadores.
trainGenerator = DataGenerator(partition["train"], labels)
valGenerator = DataGenerator(partition["validation"], labels)
testGenerator = DataGenerator(partition["test"], labels)

In [8]:
# Creamos la red y la compilamos.
modelCNN = Sequential()

modelCNN.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(180,320,3)))
modelCNN.add(Dropout(0.20))
modelCNN.add(MaxPooling2D(pool_size=(2, 2)))

modelCNN.add(Conv2D(64, kernel_size=(3, 3), activation="relu"))
modelCNN.add(Dropout(0.20))
modelCNN.add(MaxPooling2D(pool_size=(2, 2)))

modelCNN.add(Conv2D(64, kernel_size=(3, 3), activation="relu"))
modelCNN.add(Dropout(0.20))
modelCNN.add(MaxPooling2D(pool_size=(2, 2)))

modelCNN.add(Flatten())
modelCNN.add(Dense(32, activation="relu"))
modelCNN.add(Dropout(0.40))

modelCNN.add(Dense(4, activation="softmax"))

modelCNN.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

modelCNN.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 178, 318, 32)      896       
_________________________________________________________________
dropout (Dropout)            (None, 178, 318, 32)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 89, 159, 32)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 87, 157, 64)       18496     
_________________________________________________________________
dropout_1 (Dropout)          (None, 87, 157, 64)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 43, 78, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 41, 76, 64)        3

In [ ]:
# Entrenamos la red.
# El steps_per_epoch indica cuantos batches del train son un epoch.
# El validation_steps es lo mismo pero para el validation.
history = modelCNN.fit(trainGenerator, validation_data=valGenerator, use_multiprocessing=True, workers=10, steps_per_epoch=10, validation_steps=10, epochs=10)

Epoch 1/10
10/10 [==============================] - 62s 6s/step - loss: 1.6596 - accuracy: 0.2688 - val_loss: 1.3905 - val_accuracy: 0.2188
